In [3]:
import os
import ast
import torch
import numpy as np
import pandas as pd
import json
from tqdm import tqdm

In [4]:
tagged_df = pd.read_csv("./tagged_dataset/final-tags.csv")

##### Finalising tags

In [15]:
def get_prompt_from_json(image_path):
    i_uuid, ext = os.path.splitext(image_path)
    json_file_path = i_uuid + ".json"
    with open(json_file_path, "r") as json_file:
        metadata_info = json.load(json_file)
        file_prompt = metadata_info["prompt"]
    return file_prompt


In [50]:
tagged_df = pd.read_csv("./tagged_dataset/final-tags.csv")

new_updated_df = pd.DataFrame({})

for index, row in tqdm(tagged_df.iterrows()):
    row = dict(row)
    image_path = row["image_path"]
    tags = ast.literal_eval(row["tags"])
    prompt = get_prompt_from_json(image_path)
    new_tags = prompt.split(",")
    final_tags = new_tags + tags

    new_dict = row.copy()
    new_dict["tags"] = str(tags)
    new_dict["extended_tags"] = str(final_tags)
    df_row = pd.DataFrame(new_dict, index = [0])
    new_updated_df = pd.concat([new_updated_df, df_row], ignore_index = True)

0it [00:00, ?it/s]

1310it [00:00, 3479.81it/s]


In [53]:
# new_updated_df.to_csv("./tagged_dataset/final-tags.csv")

#### Adding the categories

In [33]:
tagged_df = pd.read_csv("./tagged_dataset/final-tags.csv")
tagged_df.tail()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,image_name,image_path,tags,extended_tags
1305,1305,1305,1305,5fd2e5ae-6603-4d3b-bf69-9f653bbe63e3.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"['boulder', 'pebble', 'stone']","['kept on flat paper wall', ' top view', ' foc..."
1306,1306,1306,1306,38ed39f9-178b-4265-bcb8-136154be98ba.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"['black', 'heart', 'pebble', 'stone', 'shape',...","['kept on sage floor', ' focus on product', ' ..."
1307,1307,1307,1307,88e04bed-9b57-4afb-bae1-dda030f127e7.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"['black', 'dahlia', 'flower', 'lay', 'petal', ...","['Top view of lilac paper floor', ' stones', '..."
1308,1308,1308,1308,280678d5-383f-450e-856e-08e2c11288e7.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"['circle', 'cotton candy', 'cube', 'green', 'h...","['top view of stone', ' focus on product', ' m..."
1309,1309,1309,1309,891a97af-592f-4837-bd62-693793401ff7.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"['crack', 'marble', 'stone', 'white']","['kept on old shabby', ' corrosion', ' cracked..."


In [34]:
def get_categories(image_path):
    category = image_path.split("/")[-3]
    return category

tagged_df["background_category"] = tagged_df["image_path"].apply(get_categories)

In [35]:
tagged_df = tagged_df.loc[:, ~tagged_df.columns.str.contains('^Unnamed')]
tagged_df.tail()

,image_name,image_path,tags,extended_tags,background_category
1305,5fd2e5ae-6603-4d3b-bf69-9f653bbe63e3.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"['boulder', 'pebble', 'stone']","['kept on flat paper wall', ' top view', ' foc...",MINIMALISTIC
1306,38ed39f9-178b-4265-bcb8-136154be98ba.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"['black', 'heart', 'pebble', 'stone', 'shape',...","['kept on sage floor', ' focus on product', ' ...",MINIMALISTIC
1307,88e04bed-9b57-4afb-bae1-dda030f127e7.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"['black', 'dahlia', 'flower', 'lay', 'petal', ...","['Top view of lilac paper floor', ' stones', '...",MINIMALISTIC
1308,280678d5-383f-450e-856e-08e2c11288e7.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"['circle', 'cotton candy', 'cube', 'green', 'h...","['top view of stone', ' focus on product', ' m...",MINIMALISTIC
1309,891a97af-592f-4837-bd62-693793401ff7.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"['crack', 'marble', 'stone', 'white']","['kept on old shabby', ' corrosion', ' cracked...",MINIMALISTIC


In [40]:
new_updated_df = pd.DataFrame({})

for index, row in tqdm(tagged_df.iterrows()):
    new_dict = {}
    row = dict(row)
    ext_tags = ast.literal_eval(row["extended_tags"])
    category = row["background_category"]
    if category.lower() not in ["uploaded", "generated", "podium"]:
        ext_tags.insert(0, category.lower() + " background")

    tags = ast.literal_eval(row["tags"])
    image_name = row["image_name"]
    image_path = row["image_path"]

    new_dict = {
        "image_name":image_name,
        "image_path":image_path,
        "tags":tags,
        "extended_tags":ext_tags,
        "background_category":category
    }

    new_row = pd.DataFrame([new_dict])

    new_updated_df = pd.concat([new_updated_df, new_row], ignore_index=True)

1310it [00:00, 3573.45it/s]


In [42]:
new_updated_df.tail()


,image_name,image_path,tags,extended_tags,background_category
1305,5fd2e5ae-6603-4d3b-bf69-9f653bbe63e3.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"[boulder, pebble, stone]","[minimalistic background, kept on flat paper w...",MINIMALISTIC
1306,38ed39f9-178b-4265-bcb8-136154be98ba.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"[black, heart, pebble, stone, shape, white]","[minimalistic background, kept on sage floor, ...",MINIMALISTIC
1307,88e04bed-9b57-4afb-bae1-dda030f127e7.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"[black, dahlia, flower, lay, petal, pink, ston...","[minimalistic background, Top view of lilac pa...",MINIMALISTIC
1308,280678d5-383f-450e-856e-08e2c11288e7.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"[circle, cotton candy, cube, green, heart, hea...","[minimalistic background, top view of stone, ...",MINIMALISTIC
1309,891a97af-592f-4837-bd62-693793401ff7.png,/media/pixis/pixis/Ravi_workspace/Entity-taggi...,"[crack, marble, stone, white]","[minimalistic background, kept on old shabby, ...",MINIMALISTIC


In [43]:
# new_updated_df.to_csv("./tagged_dataset/final-tags.csv")